In [ ]:
%matplotlib qt


import os, sys


import pandas as pd
import numpy as np

import scipy.io
import scipy.stats

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import seaborn as sns

import neuroseries as nts

import basefunction.anass_load as bf
import basefunction.bk_load as bk

import re
import pickle

# Helpers functions

In [2]:
def intervals_sum(intervals,units = 's'):
    to_sum = []
    for i in intervals.as_units(units).values:
        to_sum.append(np.diff(i))
    return np.sum(to_sum)

In [3]:
path = "Z:\Rat08\Rat08-20130713"


def main(path):
    
    # Initialising variables
    bk.set_current_session(path)
    
    laps = scipy.io.loadmat(bk.session+'-LapType.mat')

    danger_laps = laps['aplaps'][0]['run'][0]
    danger_laps = nts.IntervalSet(danger_laps[:,0],danger_laps[:,1],time_units = 's')

    safe_laps = laps['safelaps'][0]['run'][0]
    safe_laps = nts.IntervalSet(safe_laps[:,0],safe_laps[:,1],time_units = 's')

    laps = {'danger':danger_laps,'safe':safe_laps}

    airpuff = scipy.io.loadmat('airpuff.mat')['airpuff']
    airpuff_loc = scipy.io.loadmat(bk.session + '-PlaceCells.mat')['airP'][0][0]*137
    airpuff_t = nts.Tsd(np.array(pd.read_csv(bk.session+'.puf.evt',sep = '\t',header=None)[0]),time_units='ms')

    pos = bk.load_pos(bk.session)
    # pos_run = pos.restrict(danger_laps).as_units('s')

    # Loading Spikes Data
    
    spks,meta = bk.loadSpikeData(bk.path)
    
    #Computing mean firing rates for each kind of laps for each neurons
    FR = {}
    for laps_type in laps:
        FR.update({laps_type:[]})
        for i,n in enumerate(spks):
            tmp = n.restrict(laps[laps_type])
            fr_tmp = (tmp.times().shape[0])/(intervals_sum(laps[laps_type]))
            FR[laps_type].append(fr_tmp)
    
    # Zip everything in a dataframe
    
    df = pd.DataFrame(zip(meta),columns=['meta'])
    for i in FR:
        a = np.array(FR[i])
        df[i] = FR[i]

    
    safe_fr = np.log(df.safe)
    danger_fr = np.log(df.danger)
    df['log_safe'] = safe_fr
    df['log_danger'] = danger_fr
    
    df = df.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    
    
    # Plot
    
    
    pal = sns.cubehelix_palette(2, rot=-.5, dark=.3)

    
    plt.figure()
    sns.violinplot(data = df[['log_safe','log_danger']],palette = pal,inner = 'points')
    plt.title(bk.session)
    plt.show()
    
    print(scipy.stats.wilcoxon(df['log_safe'],df['log_danger']))
    

In [4]:
# plt.figure()

# step = 100
# for i in range(0,pos_run.shape[0],2):
#     plt.plot(pos_run['x'].iloc[i:i+step],pos_run['y'].iloc[i:i+step])
#     plt.plot(airpuff_loc,30,'o',color = 'red')
#     plt.title(pos_run.index[i])
#     plt.xlim(25,390)
#     plt.ylim(0,50)
#     plt.pause(0.001)
#     plt.cla()

In [5]:
# hpc_shank = bf.get_structure_shanks(session,'Hpc')

# hpc_mask = []
# for i in meta[:,0]:
#     hpc_mask.append(i in hpc_shank)
# hpc_neurons = spks[hpc_mask]
# meta[hpc_mask]


# bla_shank = bf.get_structure_shanks(session,'BLA')
# bla_mask = []
# for i in meta[:,0]:
#     bla_mask.append(i in bla_shank)
# # spks = spks[bla_mask]
# # meta = meta[bla_mask]

In [ ]:
bk.batch(main,verbose = True)

Loading Data from Rat08-20130708
Rat : 8 on day : 1
Working with session Rat08-20130708 @ Z:\Rat08\Rat08-20130708
Loading Rat08-20130708.clu.1


In [67]:
print(scipy.stats.wilcoxon(df['log_safe'],df['log_danger']))

WilcoxonResult(statistic=12717.0, pvalue=0.005405792663209245)


In [44]:
pal = sns.cubehelix_palette(2, rot=-.5, dark=.3)

sns.violinplot(data = df[['log_safe','log_danger']],palette = pal,inner = 'points')


In [62]:
np.where(df['safe']==0)

(array([  6,  20,  23,  24,  28,  36,  44,  48,  56,  75,  92, 193, 195,
        198, 204, 216, 217], dtype=int64),)